<a href="https://colab.research.google.com/github/LidiiaMelnyk95/phdproject/blob/main/nonBinaryLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathlib2
!pip install openpyxl==3.0.9

In [ ]:
from nltk.tokenize import RegexpTokenizer
from pathlib2 import Path
import re
import json
import os
import glob

import pandas as pd
import openpyxl
import spacy

In [ ]:
!python -m spacy download de_core_news_sm

     |████████████████████████████████| 14.9 MB 4.2 MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-py3-none-any.whl size=14907055 sha256=55c14b45319a0db8dc9aef8ba2cfbb3cdf217e87e30e724e2f5358bb8a19ea1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-eyuz0tyw/wheels/00/66/69/cb6c921610087d2cab339062345098e30a5ceb665360e7b32a
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [ ]:
!spacy download de_core_news_sm

     |████████████████████████████████| 14.9 MB 4.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [ ]:
import de_core_news_sm
nlp_de = de_core_news_sm.load()


In [ ]:
!pip install deplacy
import deplacy

In [ ]:
xls = pd.ExcelFile('/content/Non Binary.xlsx')

In [ ]:
sheet_to_df_map = {}
i = 0
for sheet_name in xls.sheet_names:
    sheet_to_df_map[sheet_name] = xls.parse(sheet_name)

In [ ]:
for key in sheet_to_df_map.keys():
  try:
    for i,row in sheet_to_df_map[key].iterrows():
      for m in str(row['Reply']).split('\n'):
        if m == 'nan':
          pass
        elif m != 'nan':
          sheet_to_df_map[key].at[i, 'Comment'] = m
  except KeyError:
    pass

In [ ]:
#for key in sheet_to_df_map.keys():
 # try:  
  #  sheet_to_df_map[key] = sheet_to_df_map[key]['Comment'].reset_index(drop = True, inplace = True)
  #except KeyError:
   # pass

In [ ]:
#sheet_to_df_map['Лист2']


In [ ]:
 #str(sheet_to_df_map['Лист2']).strip()

In [ ]:
#list_2

In [ ]:
section_delimiter = "========"
segment_separator = "========"

In [ ]:
def get_list_token():
    return "***LIST***"

def get_formula_token():
    return "***formula***"

def get_codesnipet_token():
    return "***codice***"

In [ ]:
def get_special_tokens():
    special_tokens = []
    special_tokens.append(get_list_token())
    special_tokens.append(get_formula_token())
    special_tokens.append(get_codesnipet_token())
    return 

In [ ]:
def get_separator_format(levels = None):
    level_format = '\d' if levels == None else '['+ str(levels[0]) + '-' + str(levels[1]) + ']'
    seperator_fromat = segment_separator + ',' + level_format + ",.*?\."
    return seperator_fromat


In [ ]:
def extract_sentence_words(sentence, remove_missing_emb_words = False, remove_special_tokens = False):
    if (remove_special_tokens):
        for token in get_special_tokens():
            # Can't do on sentence words because tokenizer delete '***' of tokens.
            sentence = sentence.replace(token, "")
    tokenizer = RegexpTokenizer(r'\w+')
    sentence_words = tokenizer.tokenize(sentence)
    if remove_missing_emb_words:
        sentence_words = [w for w in sentence_words if w not in missing_stop_words]

    return sentence_words

In [ ]:

def clean_section(section):
    cleaned_section = section.strip('\n')
    return cleaned_section


def get_sections_from_text(txt, high_granularity=True):
    sections_to_keep_pattern = get_separator_format() if high_granularity else get_separator_format((1, 2))
    if not high_granularity:
        # if low granularity required we should flatten segments within segment level 2
        pattern_to_ommit = get_separator_format((3, 999))
        txt = re.sub(pattern_to_ommit, "", txt)

        #delete empty lines after re.sub()
        sentences = [s for s in txt.strip().split("\n") if len(s) > 0 and s != "\n"]
        txt = '\n'.join(sentences).strip('\n')


    all_sections = re.split(sections_to_keep_pattern, txt)
    non_empty_sections = [s for s in all_sections if len(s) > 0]

    return non_empty_sections


def get_sections(key,high_granularity=True):
    sections = []
    if key in sheet_to_df_map.keys() and key != 'Лист1':
      for i,row in sheet_to_df_map[key].iterrows():
        for m in str(row['Comment']).split('\n'):
          raw_content = str(m)
          clean_txt = raw_content.strip()
          sections.append([clean_section(s) for s in get_sections_from_text(clean_txt, high_granularity)])
    return sections


      

In [ ]:

def read_youtube_df(key, n_context_sent = 1, remove_preface_segment=True, high_granularity=True):
    data = []
    targets = []
    all_sections = get_sections(key, high_granularity)
    required_sections = all_sections[1:] if remove_preface_segment and len(all_sections) > 0 else all_sections
    required_non_empty_sections = [section for section in required_sections if len(section) > 0 and section != "\n"]

    list_sentence = get_list_token() + "."
    final_sentences = []
    label = []
    for section_ind in range(len(required_non_empty_sections)):
        sentences_ = str(required_non_empty_sections[section_ind]).split('\n')
        sentences = [x for x in sentences_ if x != list_sentence]
        if sentences:
            for sentence in sentences[:-1]:
                final_sentences.append(sentence)
                label.append(0)
            final_sentences.append(sentences[-1])
            label.append(1)
    
    if len(final_sentences)>n_context_sent:
        for sent_ind in range(n_context_sent,len(final_sentences)):
            prev_context = final_sentences[sent_ind-n_context_sent:sent_ind]
            after_context = final_sentences[sent_ind: min(len(final_sentences),sent_ind+n_context_sent)]
            
            prev_context = " ".join(prev_context)
            after_context = " ".join(after_context)
            data.append([prev_context, after_context])
            targets.append(label[sent_ind-1])

    return data, targets, key
   


In [ ]:
'''
def read_youtube_df(key, n_context_sent = 1, remove_preface_segment=True, high_granularity=True):
    data = []
    targets = []
    all_sections = get_sections(key, high_granularity)
    required_sections = all_sections[1:] if remove_preface_segment and len(all_sections) > 0 else all_sections
    required_non_empty_sections = [section for section in required_sections if len(section) > 0 and section != "\n"]

    list_sentence = get_list_token() + "."
    final_sentences = []
    label = []
    for section_ind in range(len(required_non_empty_sections)):
        sentences_ = str(required_non_empty_sections[section_ind]).split('\n')
        sentences = [x for x in sentences_ if x != list_sentence]
        sentences = nlp_de(sentences)
        sentences = sentences.sents
        print(sentences)
        if sentences:
            for sentence in sentences[:-1]:
                claused_sentence = ClausedSentence(sentence.doc, sentence.start, sentence.end)
                clauses = list(claused_sentence.clauses)
                final_sentences.append('.'.join(clauses))
                label.append(0)
            final_sentences.append(sentences[-1])
            label.append(1)
    
    if len(final_sentences)>n_context_sent:
        for sent_ind in range(n_context_sent,len(final_sentences)):
            prev_context = final_sentences[sent_ind-n_context_sent:sent_ind]
            after_context = final_sentences[sent_ind: min(len(final_sentences),sent_ind+n_context_sent)]
            
            prev_context = " ".join(prev_context)
            after_context = " ".join(after_context)
            data.append([prev_context, after_context])
            targets.append(label[sent_ind-1])

    return data, targets, key
    '''

'\ndef read_youtube_df(key, n_context_sent = 1, remove_preface_segment=True, high_granularity=True):\n    data = []\n    targets = []\n    all_sections = get_sections(key, high_granularity)\n    required_sections = all_sections[1:] if remove_preface_segment and len(all_sections) > 0 else all_sections\n    required_non_empty_sections = [section for section in required_sections if len(section) > 0 and section != "\n"]\n\n    list_sentence = get_list_token() + "."\n    final_sentences = []\n    label = []\n    for section_ind in range(len(required_non_empty_sections)):\n        sentences_ = str(required_non_empty_sections[section_ind]).split(\'\n\')\n        sentences = [x for x in sentences_ if x != list_sentence]\n        sentences = nlp_de(sentences)\n        sentences = sentences.sents\n        print(sentences)\n        if sentences:\n            for sentence in sentences[:-1]:\n                claused_sentence = ClausedSentence(sentence.doc, sentence.start, sentence.end)\n           

In [ ]:
import itertools as it
import typing as tp

import spacy


VERB_POS = {"VERB", "AUX"}
FINITE_VERB_TAGS = {"VVFIN", "VMFIN", "VAFIN"}


class Clause:
    def __init__(self, spans: tp.Iterable["spacy.tokens.Span"]):
      self.spans = spans

    def __chain(self) -> tp.Iterable["spacy.tokens.Token"]:
      return [token for token in it.chain(*self.spans)]

    def __getitem__(self, index: int) -> "spacy.tokens.Token":
      return self.__chain[index]

    def __iter__(self) -> tp.Iterator:
      self.n = 0
      return self

    def __next__(self) -> "spacy.tokens.Token":
      self.n += 1
      try:
          return self[self.n - 1]
      except IndexError:
            raise StopIteration

    def __repr__(self) -> str:
        return " ".join([span.text for span in self.inner_spans])

    @property
    def is_subclause(self) -> bool:
        """Clause is a subclause iff the finite verb is in last position."""
        return (
            self[-2].tag_ in FINITE_VERB_TAGS
            if self[-1].pos_ == "PUNCT"
            else self[-1].tag_ in FINITE_VERB_TAGS
        )

    @property
    def clause_type(self) -> str:
        return "SUB" if self.is_subclause else "MAIN"

    @property
    def inner_spans(self) -> tp.List["spacy.tokens.Span"]:
        """"Spans with punctuation tokens removed from span boundaries."""
        inner_spans = []
        for span in self.spans:
            span = span[1:] if span[0].pos_ == "PUNCT" else span
            span = span[:-1] if span[-1].pos_ == "PUNCT" else span
            inner_spans.append(span)

        return inner_spans


class ClausedSentence(spacy.tokens.Span):
    """Span with extracted clause structure.

    This class is used to identify the positions of the finite verbs, to
    identify all the tokens that belong to the clause around each finite
    verb and to make a Clause object of each clause.
    """

    @property
    def __finite_verb_indices(self) -> tp.List[int]:
        return [token.i for token in self if token.tag_ in FINITE_VERB_TAGS]

    def progeny(
        self,
        index: int,
        stop_indices: tp.Optional[tp.List[int]] = None,
    ) -> tp.List["spacy.tokens.Token"]:
        """Walk trough progeny tree until a stop index is met."""
        if stop_indices is None:
            stop_indices = []

        progeny = [index]  # consider a token its own child

        for child in self[index].children:
            if child.i in stop_indices:
                continue

            progeny += [child.i] + self.progeny(child.i, stop_indices)

        return sorted(list(set(progeny)))

    @property
    def clauses(self) -> tp.Generator["Clause", None, None]:
        for verb_index in self.__finite_verb_indices:
            clause_tokens = [
                self[index]
                for index in self.progeny(
                    index=verb_index, stop_indices=self.__finite_verb_indices
                )
            ]

            spans = []

            # Create spans from range extraction of token indices
            for _, group in it.groupby(
                enumerate(clause_tokens),
                lambda index_token: index_token[0] - index_token[1].i,
            ):
                tokens = [item[1] for item in group]
                spans.append(self[tokens[0].i : tokens[-1].i + 1])

            yield Clause(spans)

doc = nlp_de('Brainwashed.... irgendwie sehen diese non binären Menschen alle gleich aus, geschorene haare, meist gefärbt. Mir kommt es eher so vor als möchten diese Menschen aufallen und ganz besonders behandelt werden.... d.h auch die extrawürste mit den ganzen pronomen. In einer Kultur die dem Individualismus strebt, ist es oft schwierig zu akzeptieren dass wir im grunde genommen einfach alle die selben sind - all one - einfach Menschen. Wieso wenden wir uns so vom natürlichen ab. Ich bin sonst ein sehr offener mensch und versuche viele Weltbilder zu verstehen, aber das finde ich so absurd. <br>Dahinter steckt wahrscheinlich das Menschliche Bedürfniss sich einer Gruppe dazugehörig zu fühlen, aufzufallen (Anerkennung) und gemocht zu werden. <br>Die meisten kiddys befinden sich noch in einer selbstfindungsphase und sind daher generell sensibel für solche themen. Ich finde es einfach fatal das einem gesunden menschen verstand solch eine, als hipp vermarktete, geisteskrankheit verkauft wird....und wenn du das hinterfragst und nicht akzeptierst, bist du ein engstirniger mensch. <br>Seid mal ehrlich. Schaut euch mal die leute an, sehen die wirklich glücklich aus. Die meisten von denen leiden an depressionen, schlucken irgendwelche pillen....mir ähnelt das ganze binary zeugs eher dem phänomen einer gespaltenen persönlichkeit, das nun schön geschmückt in unserer gesellschaft polarisiert wird. <br><br>Wir haben uns schon so viele Schubladen errichtet die uns trennen, das thema ist nur wieder eine von vielen.....')
sentences = doc.sents

for sentence in sentences:
    claused_sentence = ClausedSentence(sentence.doc, sentence.start, sentence.end)
    clauses = list(claused_sentence.clauses)
    for clause in clauses:
        print(f"{clause.clause_type}: {clause.inner_spans}")

TypeError: ignored

In [ ]:
def flatten_tree(tree):
    return ''.join([token.text_with_ws for token in list(tree)]).strip()
# Print flattened subtree of `developer`
print (flatten_tree(doc[28].subtree))

aufallen und ganz besonders behandelt werden


In [ ]:
class YoutubeDataSet():
    def __init__(self, root,n_context_sent = 1, train=True, high_granularity=False):

        root_path = sheet_to_df_map[key]
        print(root_path)
        self.textfiles = str(root_path).splitlines()

        if len(self.textfiles) == 0:
            raise RuntimeError('Found 0 images in subfolders of: {}'.format(root))
        self.train = train
        self.root = root
        self.high_granularity = high_granularity
        self.n_context_sent = n_context_sent

    def __getitem__(self, key):
        path = self.textfiles[key]

        return read_youtube_frame(Path(path),n_context_sent = self.n_context_sent,high_granularity=self.high_granularity)

    def __len__(self):
        return len(self.textfiles)

In [ ]:
comments = read_youtube_df('Лист3')

In [ ]:
comments


([["['WDRaktuell...Eine wichtige Frage ist, steht das Jo nun auf Frauen oder Männer oder Beides???']",
   "['Danke euch, mega!']"],
  ["['Danke euch, mega!']", "['Wohlstandsprobleme']"],
  ["['Wohlstandsprobleme']",
   "['... Gender, schon wieder?! Egal wo ich hinschaue und hingehe dieses Thema ist präsenter als alles Andere und ich denke mir: Wie gut muss es uns in unserer Gesellschaft gehen, dass wir uns mit so einer Scheiße auseinandersetzten müssen und das wir dadurch unsere Gesellschaft spalten und wir uns aus diesem Grund immer wieder streiten. In anderen Ländern verhungern Menschen, haben kein Dach über dem Kopf, wissen nicht ob sie den nächsten Tag oder die nächste Woche noch erleben und hier müssen wir uns damit befassen welches von den 120000 Geschlechtern wir auswählen.']"],
  ["['... Gender, schon wieder?! Egal wo ich hinschaue und hingehe dieses Thema ist präsenter als alles Andere und ich denke mir: Wie gut muss es uns in unserer Gesellschaft gehen, dass wir uns mit so ei